In [ ]:
#Load MNIST dataset
# MNIST dataset has a shape of (70000, 784) meaning there are 70,000 images with 784 dimensions (784 features).
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784")

In [ ]:
from sklearn.model_selection import train_test_split
# test_size: what proportion of original data is used for test set
train_img, test_img, train_lbl, test_lbl = train_test_split( mnist.data, mnist.target, test_size=1/7.0, random_state=0)
train_lbl[0:10]
train_img.shape

(60000, 784)

In [ ]:
import numpy as np
train_lbl = np.array([int(x) for x in train_lbl],dtype=int)
test_lbl = np.array([int(x) for x in test_lbl],dtype=int)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class TwoLayerNet(nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = nn.Linear(D_in, H)
        self.linear2 = nn.Linear(H, D_out)

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        h_sigmoid  = F.sigmoid(self.linear1(x))
        y_pred = self.linear2(h_sigmoid)

        return y_pred


In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader,TensorDataset # chỉnh sửa bên trong dùng torch 
N, D_in, H, D_out = 20, 784, 10, 10
model = TwoLayerNet(D_in, H, D_out)

criterion = nn.CrossEntropyLoss() #Calculate the error
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) 


train_img = torch.Tensor(train_img.values)
train_lbl = torch.Tensor(train_lbl).type(torch.LongTensor)
train_dataset = TensorDataset(train_img,train_lbl)
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=False) # get 10 img (10x784) to train until end: Cập nhật trọng số trên một batch lớn, phụ thuộc card mh
train_dataloader

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times Slide 15 B02

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels) # độ lệch output thực tế vs label
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,  2000] loss: 1.738
[1,  4000] loss: 1.346
[1,  6000] loss: 1.125
[2,  2000] loss: 0.979
[2,  4000] loss: 0.973
[2,  6000] loss: 0.889
Finished Training


In [ ]:
#predict 
test_img = torch.Tensor(test_img.values) # .value
outputs = model(test_img)
_, predicted = torch.max(outputs, 1)
predicted

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([0, 9, 1,  ..., 1, 3, 0])